#  Data Viz: EDA - Exploring Travel Expense reports submitted by federal instituitions
### Jesse Sallis - Bcom, MIB
#### April 2022

### Introduction: 

Purpose of notebook is to explore Canadian government officials travel expenses by utilizing advanced data visaulizations. Analysis will consist of importing/cleaning dataset, developing a few summary charts to better understand the dataset and creating informative visualizations based on initial probing questions.


### Questions that will be explored:
* How has travel expense changed over time? Are governemnt officials travelling more or less over time?
* What type of expense makes up the most of totla expense cost?
* Does longer trips lead to higher total expenses?

These questions are being asked to better understand travel habits of government officials and to start a conversation if this amount/cost is acceptable to tax payers.

### Datasets

For the purpose of this excerise, 1 datasets will be used. Data can be found here https://open.canada.ca/data/en/dataset/009f9a49-c2d9-4d29-a6d4-1a228da335ce and follows a https://open.canada.ca/en/open-government-licence-canada license.
                
* **gov_travel.csv**
    * Collection of publically recorded travel expenses from different federal instituitions
    * Columns of interest will be the ones that contain expense amount ex: airfare, total expense amount

### Data Wrangling

Let's load in our data and take a look at its properties to determine what steps are needed to tidy up data.

In [1]:
#importing required packages

import altair as alt
import pandas as pd
import numpy as np

# alt.data_transformers.enable("data_server")

#reading in dataframe 

df = pd.read_csv('data/gov_travel.csv',parse_dates=['end_date','start_date'])

#for some reason end_date is not being parsed as a datetime index

df['end_date'] = pd.to_datetime(df['end_date'],errors='coerce')

In [2]:
df.head(2)

,_id,ref_number,disclosure_group,title_en,title_fr,name,purpose_en,purpose_fr,start_date,end_date,...,airfare,other_transport,lodging,meals,other_expenses,total,additional_comments_en,additional_comments_fr,owner_org,owner_org_title
0,1,T-20120-P11-001,SLE,Chief Executive Officer,Président directeur-général,Philip Rizcallah,To attend meeting with Saskatchewan Provincial...,Pour assister à une réunion avec le gouverneme...,2020-02-03,2020-02-04,...,646.17,117.26,157.78,197.25,0.0,1118.46,NaN,NaN,casdo-ocena,Accessibility Standards Canada | Normes d’acce...
1,2,T-2020-P11-0001,SLE,Chair,Président,"Bérubé, Paul-Claude",Board members meeting,Réunion du conseil,2020-02-09,2020-02-13,...,1104.27,189.72,841.31,461.84,NaN,2597.14,NaN,NaN,casdo-ocena,Accessibility Standards Canada | Normes d’acce...


In [3]:
df.info()
print("\n")
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74573 entries, 0 to 74572
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   _id                     74573 non-null  int64         
 1   ref_number              74573 non-null  object        
 2   disclosure_group        53595 non-null  object        
 3   title_en                73865 non-null  object        
 4   title_fr                73843 non-null  object        
 5   name                    74025 non-null  object        
 6   purpose_en              74558 non-null  object        
 7   purpose_fr              74488 non-null  object        
 8   start_date              74573 non-null  datetime64[ns]
 9   end_date                73765 non-null  datetime64[ns]
 10  destination_en          74240 non-null  object        
 11  destination_fr          73927 non-null  object        
 12  airfare                 71273 non-null  float6

,_id,airfare,other_transport,lodging,meals,other_expenses,total
count,74573.000000,7.127300e+04,7.273300e+04,7.211300e+04,73373.000000,67387.000000,74384.000000
mean,37287.000000,2.572041e+13,1.671659e+02,4.031587e+02,197.954156,23.341601,1754.394517
std,21527.515149,6.866580e+15,7.143979e+03,1.203475e+04,3203.467506,121.242776,3220.345826
min,1.000000,-9.066000e+02,-7.722400e+02,0.000000e+00,0.000000,-477.200000,-906.600000
25%,18644.000000,0.000000e+00,3.000000e+01,0.000000e+00,41.750000,0.000000,481.690000
50%,37287.000000,6.442400e+02,9.582000e+01,2.000000e+02,126.100000,0.000000,1085.035000
75%,55930.000000,1.203330e+03,1.960000e+02,4.632400e+02,239.800000,12.430000,2024.377500
max,74573.000000,1.833171e+18,1.926107e+06,3.228626e+06,865324.600000,11366.910000,315315.000000


No obvious issues with the dataset, ~75k rows, 22 columns and 6 pure numeric columns

In [4]:
#seems there are some values that seems incorrect - lets find the row in question/drop row

df = df.drop(df['airfare'].idxmax())
df = df.drop(df['lodging'].idxmax())
df = df.drop(df['other_transport'].idxmax())
df = df.drop(df['meals'].idxmax())

### Exploratory Visualizations

#### Question 1: How have expenses travelled over time? Are governemnt officials travelling more or less over time?

I expect to see the pandemic to drive a sudden drop off in expenses start in Mar 2020 with a slow recovery throughout 2020/2021. Let's look at total travel expense on a monthly basis.

In [5]:
total_exp = (alt.Chart(df.set_index('start_date').groupby(pd.Grouper(freq='M')).sum().reset_index().drop(columns=['_id','total']).melt('start_date',var_name='Cost',value_name='Expense'))
    .mark_bar()
    .encode(
        alt.X("start_date",title='Date'),
        alt.Y("sum(Expense)",title="Total Expense (CAD)"),color=alt.Color('Cost')).properties(title='Total Monthly Expenses: Fig 1'))

flights = (alt.Chart(df.set_index('start_date').groupby(pd.Grouper(freq='Y')).count().loc[:'2022-01-01'].reset_index(),height=200)
    .mark_bar()
    .encode(alt.Y("start_date:T",title='Year',scale=alt.Scale(reverse=True)),
        alt.X("airfare:Q",title="Number of Flights")).properties(title='Number of Flights per Year: Fig 2'))

total_exp & flights

alt.VConcatChart(...)

Looking at the two above charts - it's clear that there was a significant increase in travel exepense/number of flights from 2004 right till 2020. As expected, travelling dropped off in 2020 due to the pandemic with the data showing travelling starting to increase throughout 2021 into 2022.

### Question 2: What type of expense makes up the most of totla expense cost?

A horizontal bar graph of 100% expense cost with expense type color for each year will be able to answer this.

In [6]:
(alt.Chart(df.set_index('start_date').groupby(pd.Grouper(freq='Y')).sum().reset_index().drop(columns=['_id','total']).melt('start_date',var_name='Cost',value_name='Expense'))
    .mark_bar(size=15)
    .encode(alt.Y("start_date:T",title='Year',scale=alt.Scale(reverse=True)),
        alt.X("sum(Expense):Q",title="Proportion of expense type",stack='normalize'),alt.Color('Cost')).properties(title='Expense Type Breakdown: Fig 3'))

alt.Chart(...)

From the above chart - its very clear - airfare makes up the largest portion of total expenses. Followed by lodging and meals

### Question 3: Does longer trips lead to higher total expenses?

We would expect this to be true. I will create a heat map of trip length vs total cost. To get a bit more detail, I will add in a faceted density plot showing trip legnth/total cost by employee type as defined in the column Disclosure Group.

In [7]:
#first we need to clean the column up

df.disclosure_group.unique()

array(['SLE', nan, 'MPSES', 'sle', 'mpses', 'FOVC'], dtype=object)

In [8]:
df['disclosure_group'] =  df.disclosure_group.fillna("Undisclosed")
df['disclosure_group'] =  df.disclosure_group.replace('sle','SLE')
df['disclosure_group'] =  df.disclosure_group.replace('mpses',"MPSES")
df.disclosure_group.unique()

array(['SLE', 'Undisclosed', 'MPSES', 'FOVC'], dtype=object)

In [9]:
#running into data quality issues where there are issues with how dates were inputed. Typos/input errors
df['trip_length'] = ((df['end_date'] - df['start_date']).dt.days).clip(0)
df = df[df.trip_length <= 30]
df = df[(df.total <= 100000) & (df.total >= 0)]

In [10]:
alt.data_transformers.disable_max_rows()
heat = alt.Chart(df).mark_rect().encode(
    alt.X('trip_length:Q', bin=alt.Bin(maxbins=20)),
    alt.Y('total:Q', bin=alt.Bin(maxbins=30)),
    alt.Color('count()')).properties(title='Trip Length vs Expense Total Heat Map: Fig 4')

heat

alt.Chart(...)

In [11]:
alt.data_transformers.disable_max_rows()
hist = (alt.Chart(df,width=200,height=200).transform_density('trip_length',groupby=['disclosure_group'],as_=['trip_length','density'],steps=200).mark_area(opacity=0.5).encode(
    alt.X('trip_length:Q'),
    alt.Y('density:Q'),color=alt.Color('disclosure_group',legend=None)).facet('disclosure_group',columns=2)).properties(title='Trip Length Density Plot: Fig 5')
hist

alt.FacetChart(...)

Looking at both the above charts, its not clear that longer trips always lead to higher total expenses. The majority of the trips are short less than 3 days and less than 2000 dollars. Theres also no meanignful difference in the trip lenth for different types of government employees.

### Concluding Remarks

For the final project, I will refine and stylize the above charts. I think I will also find a better way to visualize the above density plots - a facetted histogram will do the trick. Adding interactivty and filters will increase the usefullness of the above.

## References
* https://stackoverflow.com/questions/49817153/invert-axis-direction-altair
* https://stackoverflow.com/questions/60839767/altair-faceted-histogram-independent-scales
* https://stackoverflow.com/questions/26837998/pandas-replace-nan-with-blank-empty-string
* Module 1-5 Data Visualization